# odc-stac tutorial

This tutorial will demonstrate how to load satellite imagery using `odc-stac`. At the end of the tutorial, you will have a Cloud Optimised GeoTIFF for an area of interest.

This tutorial will cover how to:
- Set up query variables to find satellite imagery that matches
    - an area of interest
    - a date range
- Connect to a STAC catalog and find data matching the query variables
- Load the data using `odc-stac`
- Visualise and export the data


## Python imports

```
import geopandas as gpd
from odc.stac import load
from pystac_client import Client
```

## Set up query parameters

### Area of interest

```
aoi = gpd.read_file("aoi.geojson")

aoi_geometry = aoi.iloc[0].geometry
```

### Date range

```
start_date = "2021-12-24"
end_date = "2021-12-26"

date_query = (start_date, end_date)
```

### Catalog and measurements

You can use the Radiant Earth [stac-browser website](https://radiantearth.github.io/stac-browser/#/?.language=en) to preview catalog, collection, and item information.

```
catalog = "https://earth-search.aws.element84.com/v1/"
collections_query = ["sentinel-2-l2a"]
bands_query = ["red", "green", "blue"]
```


## Connect to catalog and find items

This step returns STAC items that match the query, each of which points to a Cloud Optimised GeoTIFF (COG) that contains the relevant data

In [ ]:
# Connect to the catalog
stac_client = Client.open(catalog)

### Search for items

`pystac-client` is used to search for items that match the query. The search function has a few arguments:

- `collections`: a list of collections to search
- `intersects`: an area of interest to search. Any item intersecting the geometry will be returned.
- `datetime`: a date range to search

In [ ]:
# Search for items matching the query
items = stac_client.search(
    collections=collections_query,
    intersects=aoi_geometry,
    datetime=date_query,
).item_collection()

print(f"Found {len(items)} items")

Once returned, it is possible to view relevant metadata for all discovered items.

In [ ]:
items

## Load items with odc-stac

Once the location of the items is known, `odc-stac` can load these items. The load function has a few arguments:

- `items`: a list of STAC items to load
- `bands`: a list of assets to load for each item
- `geopolygon`: the area of interest to load the data for
- `crs`: the coordinate reference system to use. `utm` will provide the most appropriate UTM projection
- `resolution`: the resolution to load the items at, in the same units as the `crs`. If using `utm` the unit is metres
- `groupby`: using `solar_day` here will merge all items captured on the same solar day into one image


```
ds_filtered = load(
    items,
    bands=bands_query,
    geopolygon=aoi_geometry,
    crs="utm",
    resolution=30,
    groupby="solar_day",
)

ds_filtered
```


## Visualise loaded data

If an `xarray` has been loaded using the Open Data Cube ecosystem, it comes with an `xarray` extension `xarray.odc` that allows you to access additional geospatial metadata and functions.

Learn more at [odc-geo](https://odc-geo.readthedocs.io/en/latest/intro-xr.html).

```
# Get a single image at the first time step, and remove the time dimension
image = ds_filtered.isel(time=0).squeeze()

image
```

```
# Create an RGB image using the .odc. extension
visualisation = image.odc.to_rgba()

# Display the RGB image on a map
visualisation.odc.explore()
```

## Export loaded data

The `.odc` extension is also useful for saving out data as Cloud Optimised GeoTIFFs (COGs).

```
visualisation.odc.write_cog(
    "sentinel2_example.tif",
    overwrite=True
)
```


## Tutorial Complete!

Try loading your saved COG in QGIS or another GIS program.